安装MindSpore框架和MindNLP套件

In [1]:
!pip install pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/unified/aarch64/mindspore-2.5.0-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 MB 112.9 MB/s eta 0:00:0000:0100:01
ERROR: Could not find a version that satisfies the requirement install (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: No matching distribution found for install


In [2]:
!pip install mindnlp==0.4.0

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install jieba

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple/

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
%env HF_ENDPOINT=https://hf-mirror.com

env: HF_ENDPOINT=https://hf-mirror.com


In [5]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn
mindspore.set_context(device_target='Ascend', device_id=0)

from mindnlp.dataset import load_dataset

from mindnlp.engine.trainer import Trainer

[WARNING] ME(250:281472944873504,MainProcess):2025-06-03-14:44:02.546.745 [mindspore/run_check/_check_version.py:329] MindSpore version 2.4.1 and Ascend AI software package (Ascend Data Center Solution)version 7.6 does not match, the version of software package expect one of ['7.3', '7.5']. Please refer to the match info on: https://www.mindspore.cn/install
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'nump

In [6]:
imdb_ds = load_dataset('imdb', split=['train', 'test'])
imdb_train = imdb_ds['train']
imdb_test = imdb_ds['test']
# 为加快运行速度只选取一部分训练
imdb_train, _ = imdb_train.split([0.1, 0.9])
imdb_test, _ = imdb_test.split([0.1, 0.9])

In [7]:
imdb_train.get_dataset_size()

2500

In [8]:
import numpy as np

def process_dataset(dataset, tokenizer, max_seq_len=512, batch_size=4, shuffle=False):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'
    def tokenize(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text, truncation=True, max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['attention_mask']

    if shuffle:
        dataset = dataset.shuffle(batch_size)

    # map dataset
    dataset = dataset.map(operations=[tokenize], input_columns="text", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=transforms.TypeCast(mindspore.int32), input_columns="label", output_columns="labels")
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                             'attention_mask': (None, 0)})

    return dataset

In [9]:
from mindnlp.transformers import OpenAIGPTTokenizer
# tokenizer
gpt_tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

# add sepcial token: <PAD>
special_tokens_dict = {
    "bos_token": "<bos>",
    "eos_token": "<eos>",
    "pad_token": "<pad>",
}
num_added_toks = gpt_tokenizer.add_special_tokens(special_tokens_dict)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
/home/mindspore/miniconda/envs/jupyter/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


In [10]:
# split train dataset into train and valid datasets
imdb_train, imdb_val = imdb_train.split([0.7, 0.3])

[WARNING] ME(250:281472944873504,MainProcess):2025-06-03-14:44:51.758.363 [mindspore/dataset/engine/datasets.py:2534] Dataset is shuffled before split.


In [11]:
dataset_train = process_dataset(imdb_train, gpt_tokenizer, shuffle=True)
dataset_val = process_dataset(imdb_val, gpt_tokenizer)
dataset_test = process_dataset(imdb_test, gpt_tokenizer)

In [12]:
next(dataset_train.create_tuple_iterator())

[Tensor(shape=[4, 512], dtype=Int64, value=
 [[  500,   246,  1322 ... 40480, 40480, 40480],
  [ 1473,   980,   246 ... 40480, 40480, 40480],
  [39516,   498,   481 ... 40480, 40480, 40480],
  [  616,   544,   808 ... 40480, 40480, 40480]]),
 Tensor(shape=[4, 512], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[4], dtype=Int32, value= [1, 1, 0, 0])]

In [13]:
import evaluate
import numpy as np
from mindnlp.engine.utils import EvalPrediction

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
from mindnlp.transformers import OpenAIGPTForSequenceClassification
from mindnlp.engine import TrainingArguments

model = OpenAIGPTForSequenceClassification.from_pretrained('openai-gpt', num_labels=2)
model.config.pad_token_id = gpt_tokenizer.pad_token_id
model.resize_token_embeddings(model.config.vocab_size + 3)

training_args = TrainingArguments(
    "./output/gpt",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=200,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(model=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, compute_metrics=compute_metrics,
                  args=training_args)

  0%|          | 0.00/457M [00:00<?, ?B/s]

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Some weights of OpenAIGPTForSequenceClassification were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 注: 如果想要运行的更快一些，可以在训练时需要V100的算力

In [15]:
# trainer.run(tgt_columns="labels")
trainer.train()

  0%|          | 0/1314 [00:00<?, ?it/s]

......{'loss': 0.5623, 'learning_rate': 1.69558599695586e-05, 'epoch': 0.46}
.{'loss': 0.5065, 'learning_rate': 1.39117199391172e-05, 'epoch': 0.91}
.

  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.38103795051574707, 'eval_accuracy': 0.8893333333333333, 'eval_runtime': 12.8526, 'eval_samples_per_second': 14.627, 'eval_steps_per_second': 14.627, 'epoch': 1.0}
{'loss': 0.3234, 'learning_rate': 1.08675799086758e-05, 'epoch': 1.37}
{'loss': 0.1771, 'learning_rate': 7.823439878234399e-06, 'epoch': 1.83}


  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.5746207237243652, 'eval_accuracy': 0.892, 'eval_runtime': 12.5869, 'eval_samples_per_second': 14.936, 'eval_steps_per_second': 14.936, 'epoch': 2.0}
{'loss': 0.1717, 'learning_rate': 4.779299847792998e-06, 'epoch': 2.28}
{'loss': 0.0905, 'learning_rate': 1.7351598173515982e-06, 'epoch': 2.74}


  0%|          | 0/188 [00:00<?, ?it/s]

{'eval_loss': 0.5379959344863892, 'eval_accuracy': 0.9013333333333333, 'eval_runtime': 12.6645, 'eval_samples_per_second': 14.845, 'eval_steps_per_second': 14.845, 'epoch': 3.0}
{'train_runtime': 544.4598, 'train_samples_per_second': 9.654, 'train_steps_per_second': 2.413, 'train_loss': 0.28584540307612544, 'epoch': 3.0}


TrainOutput(global_step=1314, training_loss=0.28584540307612544, metrics={'train_runtime': 544.4598, 'train_samples_per_second': 9.654, 'train_steps_per_second': 2.413, 'train_loss': 0.28584540307612544, 'epoch': 3.0})

In [16]:
# evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
# evaluator.run(tgt_columns="labels")
from tqdm import tqdm

def compute_accuracy(logits, labels):
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def evaluate_fn(model, test_dataset):
    total = test_dataset.get_dataset_size()
    epoch_acc = 0
    step_total = 0
    model.set_train(False)

    with tqdm(total=total) as progress_bar:
        for batch in test_dataset.create_dict_iterator():
            label = batch.pop('labels')
            logits = model(**batch).logits

            acc = compute_accuracy(logits, label)['accuracy']
            epoch_acc += acc
            
            step_total += 1
            acc=epoch_acc/step_total

    return acc

In [17]:
acc = evaluate_fn(model, dataset_val)
print(f"Accuracy: {acc}")

  0%|          | 0/188 [00:14<?, ?it/s]

Accuracy: 0.901595744680851
